In [1]:
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader

import torchvision.transforms as transforms

In [2]:
from datasets import load_dataset

ds = load_dataset("uoft-cs/cifar10")

In [3]:
label_names = ds['train'].features['label'].names
print(label_names)

['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']


In [4]:
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
def transform_images(example):
    example['img'] = transform(example['img'])
    return example


In [6]:
data_origin = ds.map(transform_images)

In [12]:
data_origin.set_format(type='torch', columns=['img', 'label'])

In [13]:
train_data = data_origin['train']
test_data = data_origin['test']

In [15]:
type(train_data[0]['img'])

torch.Tensor

In [16]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

In [32]:
image, label = train_data[0]['img'], train_data[0]['label']

In [36]:
image.size()

torch.Size([3, 32, 32])

In [46]:
class_names = ds['train'].features['label'].names

In [47]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 12, 5)    # new shape = (12, 28, 28)
        self.pool = nn.MaxPool2d(2, 2) # new shape = (12, 14, 14)
        self.conv2 = nn.Conv2d(12, 24, 5) # new shape (24, 10, 10) -> (24, 5, 5) -> Flatten (24 * 5 * 5)
        self.fc1 = nn.Linear(24 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [48]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [58]:
for epoch in range(30):
    print(f'Training epoch {epoch}...')
    
    running_loss = 0.0
    
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data['img'], data['label']
        
        optimizer.zero_grad()

        outputs = net(inputs)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Training loss {running_loss/len(train_loader):.4f}')

Training epoch 0...
Training loss 2.2968
Training epoch 1...
Training loss 2.1139
Training epoch 2...
Training loss 1.8425
Training epoch 3...
Training loss 1.6492
Training epoch 4...
Training loss 1.5437
Training epoch 5...
Training loss 1.4640
Training epoch 6...
Training loss 1.4021
Training epoch 7...
Training loss 1.3467
Training epoch 8...
Training loss 1.2979
Training epoch 9...
Training loss 1.2468
Training epoch 10...
Training loss 1.2040
Training epoch 11...
Training loss 1.1607
Training epoch 12...
Training loss 1.1260
Training epoch 13...
Training loss 1.0908
Training epoch 14...
Training loss 1.0553
Training epoch 15...
Training loss 1.0208
Training epoch 16...
Training loss 0.9971
Training epoch 17...
Training loss 0.9729
Training epoch 18...
Training loss 0.9423
Training epoch 19...
Training loss 0.9179
Training epoch 20...
Training loss 0.9039
Training epoch 21...
Training loss 0.8783
Training epoch 22...
Training loss 0.8570
Training epoch 23...
Training loss 0.8347
Tr

In [59]:
torch.save(net.state_dict(), 'trained_net.pth')

In [60]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\ccycc\AppData\Local\Temp\ipykernel_16144\1537608525.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [61]:
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in test_loader:
        images, labels = data['img'], data['label']
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
accuracy = 100 * correct / total

print(f'Accuracy of the network on the 10000 test images: {accuracy}%')

Accuracy of the network on the 10000 test images: 67.44%


In [62]:
new_transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

def load_image(image_path):
    image = Image.open(image_path)
    image = new_transform(image)
    image = image.unsqueeze(0)
    return image

In [68]:
image_paths = []
images = [load_image(img) for img in image_paths]

net.eval()
correct = 0
total = 0

net.eval()
with torch.no_grad():
    for data in images:
        images, labels = data['img'], data['label']
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        print(f'Prediction: {class_names[predicted.item()]}')